In [1]:
import re
import numpy as np
from sklearn import model_selection
from sklearn import datasets
from sklearn import svm
from sklearn.externals import  joblib
from sklearn.metrics import classification_report
from sklearn import metrics
import pandas as pd

In [2]:
df = pd.read_csv("deep-xss/data.csv")
#df=df.sample(frac=1)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,param,lable
0,aid=5&ps=DAUSUE&Keywords=%3Cscript%20src%3Dhtt...,1
1,search_string=%3Ciframe+src%3D%22http%3A%2<br/...,1
2,artistname=the%20chemical%20brothers%20-,0
3,id=2%22%3E%3Cscript%3Ealert(123)%3C/script%3E,1
4,contentid=1185,0


In [3]:
len(df)

64833

特征提取

In [4]:
featureSet = pd.DataFrame(columns=('payload','script','alert','LeftBracket','RightBracket',\
'%','isPresentDSlash','SubDir','(',')','lable'))

In [5]:
def countscript(payload):
    return payload.lower().count('script')

def countalert(payload):
    return payload.lower().count('alert')

def countLeftBracket(payload):
    return payload.count('<')

def countRightBracket(payload):
    return payload.count('>')

def countpercent(payload):
    return payload.count('%')

def isPresentDSlash(payload):
    return payload.count('\"')

def countSubDir(payload):
    return payload.count('\'')

def countLeft(payload):
    return payload.count('(')

def countRight(payload):
    return payload.count(')')

In [8]:
def getFeature(payload,lable):
    fea = []
    payload = str(payload)
        
    fea.append(payload)
    fea.append(countscript(payload))
    fea.append(countalert(payload))   
    fea.append(countLeftBracket(payload))
    fea.append(countRightBracket(payload))
    fea.append(countpercent(payload))
    fea.append(isPresentDSlash(payload))
    fea.append(countSubDir(payload))
    fea.append(countLeft(payload))
    fea.append(countRight(payload))
    fea.append(str(lable))
    
    return fea

In [9]:
for i in range(len(df)):
    features = getFeature(df["param"].loc[i],df["lable"].loc[i])
    featureSet.loc[i] = features

In [10]:
featureSet.head()

,payload,script,alert,LeftBracket,RightBracket,%,isPresentDSlash,SubDir,(,),lable
0,aid=5&ps=DAUSUE&Keywords=%3Cscript%20src%3Dhtt...,2,0,1,1,7,0,0,0,0,1
1,search_string=%3Ciframe+src%3D%22http%3A%2<br/...,0,0,1,1,9,0,0,0,0,1
2,artistname=the%20chemical%20brothers%20-,0,0,0,0,3,0,0,0,0,0
3,id=2%22%3E%3Cscript%3Ealert(123)%3C/script%3E,2,1,0,0,6,0,0,1,1,1
4,contentid=1185,0,0,0,0,0,0,0,0,0,0


In [11]:
featureSet.to_csv("deep-xss/features.csv",index=False)

训练数据集

In [12]:
len(featureSet)

64833

In [14]:
featureSet.groupby(featureSet['lable']).size()

lable
0    31407
1    33426
dtype: int64

In [17]:
X = featureSet.drop(['payload','lable'],axis=1).values
y = featureSet['lable'].values

In [20]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y ,test_size=0.3)

In [19]:
import pandas as pd
from sklearn import tree,cross_validation
import sklearn.ensemble as ek
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

E:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [21]:
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         "GNB":GaussianNB(),
         "LogisticRegression":LogisticRegression()   
}

In [22]:
#测试不同算法的准确度
results = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,y_train)
    score_test = clf.score(X_test,y_test)
    score_train = clf.score(X_train,y_train)
    print ("%s : %s %s" %(algo, score_test,score_train))
    results[algo] = score_test

DecisionTree : 0.9827249357326479 0.9818213868629222
RandomForest : 0.9827763496143959 0.9818213868629222
Adaboost : 0.9827763496143959 0.9816891787673798
GradientBoosting : 0.9822107969151671 0.9815129013066567
GNB : 0.9786632390745501 0.9768415485974925
LogisticRegression : 0.9811825192802056 0.9800145428905097


In [23]:
print(results)

{'DecisionTree': 0.9827249357326479, 'RandomForest': 0.9827763496143959, 'Adaboost': 0.9827763496143959, 'GradientBoosting': 0.9822107969151671, 'GNB': 0.9786632390745501, 'LogisticRegression': 0.9811825192802056}


In [24]:
winner = max(results, key=results.get)
print(winner)

RandomForest


In [25]:
clf = model[winner]
res = clf.predict(X_test)
mt = confusion_matrix(y_test, res)#混淆矩阵
print("False positive rate : %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate : %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

False positive rate : 0.254561 %
False negative rate : 3.103173 %


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [27]:
#准确率 = 分类正确的样本数/总样本数
accuracy_score(y_test, res)

0.9827763496143959

In [38]:
result = pd.DataFrame(columns=('payload','script','alert','LeftBracket','RightBracket',\
'%','isPresentDSlash','SubDir','(',')','lable'))

results = getFeature('Itemid=%22onmouseover=alert%28document.cookie%29%20bad=%22', '1')
result.loc[0] = results
result = result.drop(['payload','lable'],axis=1).values
print(clf.predict(result))

['1']
